In [32]:
library("coloc")
suppressMessages(suppressWarnings(library(tidyverse)))
library(locuscomparer)
library(susieR)
set.seed(1)
suppressMessages(library('Rfast'))

In [33]:
## TRANS

In [34]:
df_gwas <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/gwas/",pattern='.z',full.names = TRUE)
df_gwas <- df_gwas[ !grepl(".bgen.bcor.ld", df_gwas) ]
df_eqtl <- list.files("../../ANALYSIS/COLOC_SUSIE/harmonized_eur/eqtl/",pattern='.z',full.names = TRUE)
df_eqtl <- df_eqtl[ !grepl(".bgen.bcor.ld", df_eqtl) ]

In [35]:
pip_df <- data.frame()
suppressMessages(suppressWarnings(
for (i in seq_along(df_gwas))
{
tryCatch({
# tissue
tissue <- tail(str_split(df_gwas,"/|.z")[[i]],n=3)[2]
    # gwas
s_gwas <- read.table(df_gwas[i],header=T,sep=" ")
z_gwas=s_gwas$beta/s_gwas$se
R_gwas <- as.matrix(read.table(paste0(df_gwas[i],".bgen.bcor.ld"),header=FALSE))
fitted_gwas <- susie_rss(z_gwas,R_gwas,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
# eqtl
s_eqtl <- read.table(df_eqtl[i],header=T,sep=" ")
z_eqtl=s_eqtl$beta/s_eqtl$se
R_eqtl <- as.matrix(read.table(paste0(df_eqtl[i],".bgen.bcor.ld"),header=FALSE))
fitted_eqtl <- susie_rss(z_eqtl,R_eqtl,estimate_prior_variance = FALSE,max_iter = 5000,L=10,coverage=0.90)
    
## coloc and susie
if(requireNamespace("susieR",quietly=TRUE)) {susie.res=coloc.susie(fitted_gwas,fitted_eqtl)}
}, error=function(e){})
## data frame
if (length(s_gwas$rsid)==length(susie.res$results$SNP.PP.H4.row4)){

df <- data.frame(MARKERNAME=s_gwas$rsid,H3_abf=susie.res$results$SNP.PP.H4.row3,H4_abf=susie.res$results$SNP.PP.H4.row4,eqtl_chr=s_eqtl$chromosome,eqtl_pos=s_eqtl$position)%>%mutate(Tissue=tissue)
pip_df   <- rbind(pip_df ,df)
    }
    }
))

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  3.98e-05 
[1] "PP abf for shared variant: 0.00398%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.64900   0.34600   0.00501 
[1] "PP abf for shared variant: 0.501%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  8.50e-05 
[1] "PP abf for shared variant: 0.0085%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.65000   0.34600   0.00486 
[1] "PP abf for shared variant: 0.486%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  2.60e-10 
[1] "PP abf for shared variant: 2.6e-08%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
  0.00000   0.00000   0.64800   0.34500   0.00691 
[1] "PP abf for shared variant: 0.691%"
PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.00e+00  0.00e+00  4.83e-20  1.00e+00  2.11e-07 
[1] "PP abf for shared variant: 2.11e-05%

In [36]:
df_smr <- list.files("../../ANALYSIS/SMR/eur_IV/",pattern='*.smr',full.names = TRUE)
smr_results <- data.frame()

for (i in 1:length(df_smr))
{
    name <-  tail(str_split(df_smr,"/|.smr")[[i]],n=2)[1]
df  <- read.table(df_smr[i],header=TRUE,fill = TRUE)%>%mutate(filename=name)%>%distinct()
smr_results<- rbind(smr_results,df)}
smr_df <- arrange(smr_results%>%filter(p_SMR < 1E-04 & p_HEIDI >= 0.05)%>%select(probeID,Gene,topSNP,topSNP_chr,topSNP_bp,Freq,p_eQTL,p_GWAS,p_SMR ,p_HEIDI,nsnp_HEIDI,filename),topSNP_chr,topSNP_bp)%>%
distinct()%>%rename(MARKERNAME=topSNP)

In [37]:
a<- merge(pip_df,smr_df,by="MARKERNAME")
a$Tissue <- str_split_fixed(a$Tissue, '_rs', 2)[,1]
a$filename <- str_split_fixed(a$filename, '_rs', 2)[,1]

In [41]:
a%>%filter(Tissue==filename)%>%distinct(MARKERNAME,Tissue,probeID,.keep_all =TRUE)

MARKERNAME H3_abf        H4_abf        eqtl_chr eqtl_pos 
1  rs1010554   0.000000e+00  0.000000e+00  3        52542919
2  rs11043288  7.656418e-56  1.422313e-63 12       122451187
3  rs11043288  0.000000e+00  0.000000e+00 12       122451187
4  rs12303671  0.000000e+00  0.000000e+00 12       124492610
5  rs12489828  0.000000e+00  0.000000e+00  3        52567014
6  rs12489828  0.000000e+00  0.000000e+00  3        52567014
7  rs12489828  0.000000e+00  0.000000e+00  3        52567014
8  rs1354091   1.146341e-17 3.720051e-101  3       186505900
9  rs2083180   0.000000e+00  0.000000e+00  3        52668119
10 rs222843    6.305023e-46  2.591120e-08 17         7145981
11 rs222843    9.292642e-02  3.975397e-04 17         7145981
12 rs222843   1.421060e-120  0.000000e+00 17         7145981
13 rs222843    9.613558e-48 1.409186e-127 17         7145981
14 rs222851    1.797042e-06  4.132581e-03 17         7139238
15 rs222851    1.797042e-06  4.132581e-03 17         7139238
16 rs222851    4.812288e-78 9.472076e-153 17         7139238
17 rs222851    6.206052e-04  1.716672e-12 17         7139238
18 rs222851    0.000000e+00  0.000000e+00 17         7139238
19 rs222852    1.920505e-94 5.597130e-161 17         7140606
20 rs222852    0.000000e+00  0.000000e+00 17         7140606
21 rs222852    1.253420e-42  9.385406e-07 17         7140606
22 rs222857    3.333130e-02  1.856505e-11 17         7164563
23 rs222857    1.244194e-42  7.618630e-07 17         7164563
24 rs2293243   8.797650e-01  6.332210e-23  3       186522737
25 rs2604209   7.296519e-42  1.305550e-04  4        89621072
26 rs2604209   2.265056e-07  6.567207e-06  4        89621072
27 rs2657880   3.586017e-70  0.000000e+00 12        56863770
28 rs352162    0.000000e+00  0.000000e+00  3        52252969
29 rs4765219   0.000000e+00  0.000000e+00 12       124440110
30 rs4765528   0.000000e+00  0.000000e+00 12       124462254
31 rs4930724   0.000000e+00  0.000000e+00 12       124423817
32 rs4930726   0.000000e+00  0.000000e+00 12       124428331
33 rs4930726   0.000000e+00  0.000000e+00 12       124428331
34 rs5412      5.544498e-01  6.249647e-08 17         7184046
35 rs7133378   0.000000e+00  0.000000e+00 12       124409502
36 rs7133378   9.479073e-64  1.531786e-49 12       124409502
37 rs7134375   1.446954e-04  3.057208e-07 12        20473758
38 rs7958691   0.000000e+00  0.000000e+00 12       124440743
39 rs7958691   0.000000e+00  0.000000e+00 12       124440743
40 rs7958691   0.000000e+00  0.000000e+00 12       124440743
41 rs7958691   0.000000e+00  0.000000e+00 12       124440743
42 rs7964945   0.000000e+00  0.000000e+00 12       124437668
43 rs8046873   1.003702e-01  5.114743e-08 16        82751932
44 rs863750    0.000000e+00  0.000000e+00 12       124505444
45 rs863750    0.000000e+00  0.000000e+00 12       124505444
46 rs9879090   0.000000e+00  0.000000e+00  3        52648265
47 rs9971695   0.000000e+00  0.000000e+00 12       124413491
48 rs9971695   5.492403e-25  0.000000e+00 12       124413491
   Tissue                   probeID         Gene          topSNP_chr topSNP_bp
1  Artery_Aorta             ENSG00000168268 NT5DC2         3          52542919
2  Muscle_Skeletal          ENSG00000110987 BCL7A         12         122451187
3  Heart_Atrial_Appendage   ENSG00000158023 WDR66         12         122451187
4  Heart_Atrial_Appendage   ENSG00000270028 RP11-380L11.4 12         124492610
5  Adipose_Subcutaneous     ENSG00000168268 NT5DC2         3          52567014
6  Adipose_Visceral_Omentum ENSG00000168268 NT5DC2         3          52567014
7  Artery_Tibial            ENSG00000168268 NT5DC2         3          52567014
8  Adipose_Subcutaneous     ENSG00000163918 RFC4           3         186505900
9  Artery_Coronary          ENSG00000163938 GNL3           3          52668119
10 Artery_Tibial            ENSG00000170291 ELP5          17           7145981
11 Thyroid                  ENSG00000181885 CLDN7         17           7145981
12 Whole_Blood              ENSG00000170291 ELP5          17      